In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import os
from torch.utils import data

NUM_EPOCHS = 1200000

# 定义回归模型（需要选手自己设计填写）
# Define the regression model (participants need to design and fill it in themselves)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.w = nn.Parameter(torch.ones(2, dtype=torch.float64), requires_grad=True)
        # self.b = nn.Parameter(torch.zeros(1, dtype=torch.float64), requires_grad=True)

    def forward(self, f) -> torch.Tensor:
        return torch.matmul(f, self.w)


def train(a_data):
    a_val = []
    features = []
    labels = []
    for i in range(len(a_data)):
        a_val.append(a_data["a_seq"][i])

    for i in range(3, len(a_data)):
        features.append([a_val[i - 2], a_val[i - 3]])
        labels.append([a_val[i]])

    features = torch.tensor(features, dtype=torch.float64)
    labels = torch.tensor(labels, dtype=torch.float64)

    dataset = data.TensorDataset(*(features, labels))
    data_iter = data.DataLoader(dataset, batch_size=4, shuffle=True)

    net = Net()
    loss = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.01)

    for epoch in range(NUM_EPOCHS):
        # print(features, labels)
        l = loss(net(features), labels.T[0])
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        # for x, y in data_iter:
        #     l = loss(net(x), y.T[0])
        #     optimizer.zero_grad()
        #     l.backward()
        #     optimizer.step()

    return net

# 进行回归的函数（需要选手自己设计填写）
# Function for parameter solving (participants need to design and fill it in themselves)
def pqregression(a_data):
    net = train(a_data)
    p_v = net.w[0].item()
    q_v = -1 * net.w[1].item()
    return p_v, q_v

In [ ]:
if __name__ == '__main__':
    #-------------读取训练集,训练集地址已经设定好，下面这段不用修改------------------#
    #-----Read the training set, the address of the training set has been set, and the following section does not need to be modified-------#
    train_path = "/bohr/train-btk3/v1/a_seq_train.csv"
    data_train = pd.read_csv(train_path)

    #-------------读取测试集---------------#“DATA_PATH”是测试集加密后的环境变量，按照如下方式可以在提交后，系统评分时访问测试集，但是选手无法直接下载
    #----Read the testing set, “DATA_PATH” is an environment variable for the encrypted test set. After submission, you can access the test set for system scoring in the following manner, but the contestant cannot download it directly.-----#
    if os.environ.get('DATA_PATH'):
        DATA_PATH = os.environ.get("DATA_PATH") + "/"
    else:
        print("Baseline运行时，因为无法读取测试集，所以会有此条报错，属于正常现象")
        print("When baseline is running, this error message will appear because the test set cannot be read, which is a normal phenomenon.")
        #Baseline运行时，因为无法读取测试集，所以会有此条报错，属于正常现象
        #When baseline is running, this error message will appear because the test set cannot be read, which is a normal phenomenon.
    testA_path = DATA_PATH + "a_seq_testA.csv"  #读取测试集A, read testing setA
    data_testA = pd.read_csv(testA_path)
    testB_path = DATA_PATH + "a_seq_testB.csv" #读取测试集B,read teseting setB
    data_testB = pd.read_csv(testB_path)
    #--------------开始进行模型回归--Start Parameter Solving-------------#
    p_train, q_train = pqregression(data_train)
    p_testA, q_testA = pqregression(data_testA)
    p_testB, q_testB = pqregression(data_testB)
    p = [p_train,p_testA,p_testB]
    q = [q_train,q_testA,q_testB]
    #-----保存参数p，q到CSV文件到submission.csv, to save the parameters p and q to the .csv file-----#
    df_params = pd.DataFrame({'p': p, 'q': q})
    print(df_params)
    csv_file_path = 'submission.csv'
    df_params.to_csv(csv_file_path, index=False)